In [93]:
#import das biblioteca
import pandas as pd
import requests
import json

In [94]:
#definição url API
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'


In [95]:
#lendo arquivo com IDs e importando os dados
df = pd.read_csv('santander_example.csv')
user_ids = df['UserID'].tolist()
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]

In [96]:
#cria classes a partir dos dados

class Account:
    def __init__(self, id, number, agency, balance, limit):
        self.id = id
        self.number = number
        self.agency = agency
        self.balance = balance
        self.limit = limit

class Card:
    def __init__(self, id, number, limit):
        self.id = id
        self.number = number
        self.limit = limit

class Person:
    def __init__(self, id, name, account, card):
        self.id = id
        self.name = name
        self.account = account
        self.card = card

In [97]:
#cria uma lista de objetos com os usuarios retiradas da lista de IDs
people = []
for person_data in users:
    account_data = person_data['account']
    card_data = person_data['card']
    
    account = Account(**account_data)
    card = Card(**card_data)
    
    person = Person(id=person_data['id'], name=person_data['name'], account=account, card=card)
    people.append(person)

In [98]:
#inicializa listas para transformar os dados em dataframe
ids = []
names = []
account_ids = []
account_numbers = []
account_agencies = []
account_balances = []
account_limits = []
card_ids = []
card_numbers = []
card_limits = []

In [99]:
#popula as listas com os dados
for person in people:
    ids.append(person.id)
    names.append(person.name)
    account_ids.append(person.account.id)
    account_numbers.append(person.account.number)
    account_agencies.append(person.account.agency)
    account_balances.append(person.account.balance)
    account_limits.append(person.account.limit)
    card_ids.append(person.card.id)
    card_numbers.append(person.card.number)
    card_limits.append(person.card.limit)

In [100]:
#transforma os dados no formato dicionario
data = {
    'ID': ids,
    'Name': names,
    'Account ID': account_ids,
    'Account Number': account_numbers,
    'Account Agency': account_agencies,
    'Account Balance': account_balances,
    'Account Limit': account_limits,
    'Card ID': card_ids,
    'Card Number': card_numbers,
    'Card Limit': card_limits
}

In [101]:
#cria o dataframe de output e exporta
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)